In [0]:
create schema if not exists eclerx.E_com;
use schema e_com;

-- 1) DATE DIMENSION: 30 days for June 2025

CREATE OR REPLACE TABLE dim_date (
 date_key DATE,
 day_of_month INT,
 month_of_year INT,
 year_val INT,
 weekday STRING
);
INSERT INTO dim_date
SELECT
 d AS date_key,
 day(d) AS day_of_month,
 month(d) AS month_of_year,
 year(d) AS year_val,
 date_format(d,'EEEE') AS weekday
FROM (
 SELECT explode(sequence(to_date('2025-06-01'), to_date('2025-06-30'), interval 1 day)) AS d
);

-- 2) CUSTOMER DIMENSION: 20 customers across 4 regions & segments
CREATE OR REPLACE TABLE dim_customer (
 customer_id INT,
 customer_name STRING,
 region STRING,
 segment STRING
);
INSERT INTO dim_customer VALUES
 (1,  'Alice Johnson',   'North',    'Consumer'),
 (2,  'Bob Smith',       'South',    'Corporate'),
 (3,  'Carol Lee',       'East',     'Home Office'),
 (4,  'David Kim',       'West',     'Consumer'),
 (5,  'Eva Martinez',    'North',    'Corporate'),
 (6,  'Frank Wang',      'South',    'Home Office'),
 (7,  'Grace Singh',     'East',     'Consumer'),
 (8,  'Hector Ruiz',     'West',     'Corporate'),
 (9,  'Irene Zhao',      'North',    'Home Office'),
 (10, 'James O\'Connor', 'South',    'Consumer'),
 (11, 'Karen Patel',     'East',     'Corporate'),
 (12, 'Leo Müller',      'West',     'Home Office'),
 (13, 'Mona Fernandes',  'North',    'Consumer'),
 (14, 'Nate Brown',      'South',    'Corporate'),
 (15, 'Olivia Davis',    'East',     'Home Office'),
 (16, 'Paul García',     'West',     'Consumer'),
 (17, 'Quinn Allen',     'North',    'Corporate'),
 (18, 'Rita Khan',       'South',    'Home Office'),
 (19, 'Sam Wilson',      'East',     'Consumer'),
 (20, 'Tina Yang',       'West',     'Corporate');

-- 3) PRODUCT DIMENSION: 20 products in 5 categories

CREATE OR REPLACE TABLE dim_product (
 product_id INT,
 product_name STRING,
 category STRING
);
INSERT INTO dim_product VALUES
 (1,  'UltraWidget',      'Widgets'),
 (2,  'MegaWidget',       'Widgets'),
 (3,  'SuperGadget',      'Gadgets'),
 (4,  'HyperGadget',      'Gadgets'),
 (5,  'PowerTool',        'Tools'),
 (6,  'SpeedTool',        'Tools'),
 (7,  'SmartPhone X',     'Electronics'),
 (8,  'SmartPhone Y',     'Electronics'),
 (9,  'Laptop Pro',       'Computers'),
 (10, 'Laptop Air',       'Computers'),
 (11, 'Desk Lamp',        'Office'),
 (12, 'Office Chair',     'Office'),
 (13, 'Wireless Mouse',   'Accessories'),
 (14, 'Keyboard K100',    'Accessories'),
 (15, 'Action Camera',    'Cameras'),
 (16, 'DSLR Camera',      'Cameras'),
 (17, 'Bluetooth Speaker','Audio'),
 (18, 'Noise Cancelling Headphones','Audio'),
 (19, 'Fitness Tracker',  'Wearables'),
 (20, 'Smart Watch',      'Wearables');

 -- 4) FACT SALES:

CREATE OR REPLACE TABLE fact_sales (
 sale_id BIGINT,
 date_key DATE,
 customer_id INT,
 product_id INT,
 quantity INT,
 unit_price DECIMAL(10,2),
 total_amount DECIMAL(12,2)
);


INSERT INTO fact_sales
SELECT
 t.id                                   AS sale_id,
 d.date_key                             AS date_key,
 c.customer_id                          AS customer_id,
 p.product_id                           AS product_id,
 CAST( round(rand() * 9 + 1 ) AS INT )  AS quantity,
 ROUND(rand() * 95 + 5, 2)              AS unit_price,
 ROUND( (CAST(round(rand() * 9 + 1) AS INT)) * (ROUND(rand() * 95 + 5,2)), 2) AS total_amount
FROM (
 -- generate IDs 1 through 80
 SELECT id FROM range(1, 81)
) t
-- pick a date by cycling through the 30 dim_date rows
LEFT JOIN (
 SELECT date_key, ROW_NUMBER() OVER (ORDER BY date_key) AS rn
 FROM dim_date
) d ON d.rn = pmod(t.id - 1, 30) + 1
-- pick a customer by cycling through the 20 dim_customer rows
LEFT JOIN (
 SELECT customer_id, ROW_NUMBER() OVER (ORDER BY customer_id) AS rn
 FROM dim_customer
) c ON c.rn = pmod(t.id - 1, 20) + 1
-- pick a product by cycling through the 20 dim_product rows
LEFT JOIN (
 SELECT product_id, ROW_NUMBER() OVER (ORDER BY product_id) AS rn
 FROM dim_product
) p ON p.rn = pmod(t.id - 1, 20) + 1;

--Task One Total sales by region

select dc.region,sum(fs.total_amount) as total_sales from eclerx.e_com.dim_customer as dc
join eclerx.e_com.fact_sales as fs on dc.customer_id = fs.customer_id
group by dc.region
order by total_sales desc;

--Top 5 products by total revenue

select dp.product_name,sum(fs.total_amount) as total_sales from eclerx.e_com.dim_product as dp
join eclerx.e_com.fact_sales as fs on dp.product_id = fs.product_id
group by dp.product_name
order by total_sales desc
LIMIT 5;

-- Number of orders and average order value by customer segment

select dc.segment,count(distinct fs.sale_id) as total_orders,avg(fs.total_amount) from eclerx.e_com.dim_customer as dc
join eclerx.e_com.fact_sales as fs on dc.customer_id = fs.customer_id
group by dc.segment
order by total_orders desc;

-- Monthly sales total using dim_date

select dd.month_of_year,sum(fs.total_amount) as total_sales from eclerx.e_com.dim_date as dd
join eclerx.e_com.fact_sales as fs on dd.date_key = fs.date_key
group by dd.month_of_year
order by total_sales desc;

select * from eclerx.e_com.dim_date;

--Task 5: Running total of sales per region (Window function)

select region,total_sales,sum(total_sales) over (partition by region order by total_sales desc) as Running_total from 








